# Nepali Text Autocorrection System

### Author: [Anish Shilpakar](https://github.com/JuJu2181)

## About this project
Text autocorrection system is a software application that is designed to automatically correct errors in text input by suggesting the most appropriate corrections. The system uses algorithms to analyze the input text and compare it with a database of correct words, phrases or sentences, and provides suggestions for corrections in real-time.

Text autocorrection systems are essential tools for improving the accuracy and efficiency of text-based communication, especially in situations where the input is prone to errors. For example, in mobile devices, text messages, emails, and other applications, text autocorrection can significantly reduce the time required to type and edit text, making it easier and faster for users to communicate.

The scope of text autocorrection systems is vast, as it can be used in various applications, including but not limited to social media, instant messaging, search engines, email clients, and document editors. Additionally, text autocorrection systems can be implemented in different languages, making them beneficial for users worldwide, irrespective of their language proficiency. 

In this project I have implemented a simple text autocorrection system for Nepali language. For this project, I have taken this dataset [Nepali words dictionary](https://www.kaggle.com/datasets/sangamthapa/nepali-dictionary) from Kaggle. This Nepali text autocorrection system utilizes Jaccard Similarity and Levenshtein Distance algorithms to suggest the most similar words to a given input word. By leveraging these algorithms, the system provides accurate and efficient corrections to Nepali text, enhancing the overall user experience. As this system is still in development, it may not be accurate as of now, but it will be improved as time moves on. 

### Programming Language Used
- Python

> If you want to contribute to this project, feel free to fork this project and send pull requests for your contributions. And if you like this project, don't forget to leave a ⭐ in this repository




Necessary imports

In [1]:
import re 
import pandas as pd 
import numpy as np 
import textdistance
from collections import Counter

Finding the number of words and creating unique vocabulary from the input txt file

In [49]:
words = []
with open('./nepali_dict_words.txt','r',encoding='utf-8') as f:
    data = f.read()
    words = data.split()
vocabulary = set(words)
print(f"Total Words: {len(words)}")
print(f"Total unique words in vocabulary: {len(vocabulary)}")
print(f"First 10 words:\n{words[:10]}")
print(f"Last 10 words:\n{words[-10:]}")

Total Words: 556344
Total unique words in vocabulary: 496265
First 10 words:
['अ', 'पत्रकारको', 'अँ', 'छौँ', 'अँकाइ', 'अँगरखा', 'अँगर्खा', 'अँगार', 'अँगाल्–नु', 'अँगाले']
Last 10 words:
['पढ्छ', 'लेख्छु', 'लेख्छ', 'लेख्छन', 'लेख्छिन', 'पढ्छिन', 'सीता', 'विद्यालय', 'जानुहुन्छ', 'पुगे']


In [50]:
print('कमल' in words)

True


Finding the frequency of words from the list of words

In [51]:
word_freq = {}
for word in words:
    if word not in word_freq.keys():
        word_freq[word] = 1
    else:
        word_freq[word] += 1

In [52]:
word_freq_items = list(word_freq.items())
word_freq_items[0:10]

[('अ', 3),
 ('पत्रकारको', 3),
 ('अँ', 4),
 ('छौँ', 3),
 ('अँकाइ', 3),
 ('अँगरखा', 3),
 ('अँगर्खा', 3),
 ('अँगार', 3),
 ('अँगाल्–नु', 3),
 ('अँगाले', 3)]

Calculating probabilities based on frequency

In [53]:
probs = {}
Total = sum(word_freq.values())
for word in word_freq.keys():
    probs[word] = word_freq[word]/Total 
list(probs.items())[0:10]

[('अ', 5.392347180880894e-06),
 ('पत्रकारको', 5.392347180880894e-06),
 ('अँ', 7.189796241174525e-06),
 ('छौँ', 5.392347180880894e-06),
 ('अँकाइ', 5.392347180880894e-06),
 ('अँगरखा', 5.392347180880894e-06),
 ('अँगर्खा', 5.392347180880894e-06),
 ('अँगार', 5.392347180880894e-06),
 ('अँगाल्–नु', 5.392347180880894e-06),
 ('अँगाले', 5.392347180880894e-06)]

Implementing an autocorrect function using textdistance package

In [60]:
def my_autocorrect(input_word, n = 5, dist="jac"):
    """
    **Description**
    This function will return top n similar words that could be correction for given word
    - Similarity can either be based on Jaccard similarity or based on Levenshtein distance

    **Arguments** 
    input_word: word to be corrected
    n: no of corrections suggested for input_word
    dist: similarity to be used. Use "jac" for Jaccard Similarity and "lev" for Levenshtein distance

    **Returns**
    A dataframe showing the top n suggestions along with their probability and similarity
    """
    if input_word in vocabulary:
        print("Word is correct, so no correction needed")
        return input_word
    else: 
        if dist == "jac":
            sim = [1-(textdistance.Jaccard(qval=2).distance(v,input_word)) for v in word_freq.keys()]
        elif dist == "lev":
            sim = [1-(textdistance.levenshtein(v,input_word)) for v in word_freq.keys()]
        df = pd.DataFrame.from_dict(probs,orient='index').reset_index() 
        df = df.rename(columns={'index':'Word',0:'Prob'})
        df['Similarity'] = sim 
        output = df.sort_values(['Similarity','Prob'],ascending=False).head(n)
        # return list(output['Word'])[0]
        return output

Performing some tests

In [55]:
my_autocorrect("नेपाब", n = 10, dist="jac")

,Word,Prob,Similarity
87235,नेपा,0.000002,0.750000
18743,नेपाल,0.000020,0.600000
80415,बनेपा,0.000002,0.600000
261203,नेपार,0.000002,0.600000
380515,नेपाः,0.000002,0.600000
206423,बनेपाबाट,0.000002,0.571429
18744,नेपाली,0.000014,0.500000
18745,नेपाले,0.000005,0.500000
42546,नेपालाबाट,0.000002,0.500000
74073,सानेपा,0.000002,0.500000


In [56]:
my_autocorrect("नेपाब", n = 10, dist="lev")

,Word,Prob,Similarity
18743,नेपाल,0.000020,0
87235,नेपा,0.000002,0
261203,नेपार,0.000002,0
380515,नेपाः,0.000002,0
18734,नेता,0.000040,-1
18744,नेपाली,0.000014,-1
18767,नेवा,0.000007,-1
10685,चेपाइ,0.000005,-1
10688,चेपाङ,0.000005,-1
15244,तेजाब,0.000005,-1


In [57]:
my_autocorrect("कमलृ")

,Word,Prob,Similarity
4841,कमल,0.000005,0.666667
4848,कमलो,0.000007,0.500000
4842,कमला,0.000005,0.500000
79094,कमले,0.000002,0.500000
190737,कमली,0.000002,0.500000


In [58]:
my_autocorrect("कमलृ", dist="lev")

,Word,Prob,Similarity
4848,कमलो,0.000007,0
4841,कमल,0.000005,0
4842,कमला,0.000005,0
79094,कमले,0.000002,0
190737,कमली,0.000002,0


In [61]:
my_autocorrect("कम")

Word is correct, so no correction needed


'कम'

In [62]:
my_autocorrect("क")

Word is correct, so no correction needed


'क'

In [63]:
my_autocorrect("सबइ")

,Word,Prob,Similarity
32097,सब,0.000009,0.500000
32099,सबल,0.000005,0.333333
32108,सबै,0.000005,0.333333
22807,बइन,0.000004,0.333333
100861,सबक,0.000002,0.333333


In [64]:
my_autocorrect("सबइ",dist="lev")

,Word,Prob,Similarity
32097,सब,0.000009,0
32099,सबल,0.000005,0
32108,सबै,0.000005,0
32746,साइ,0.000005,0
100861,सबक,0.000002,0


In [65]:
my_autocorrect("नमा")

,Word,Prob,Similarity
17706,नमाज,0.000005,0.666667
36848,ऐनमा,0.000002,0.666667
37569,रनमा,0.000002,0.666667
47428,उनमा,0.000002,0.666667
50514,मनमा,0.000002,0.666667


In [66]:
my_autocorrect("नमा",dist="lev")

,Word,Prob,Similarity
2644,आमा,0.000011,0
3804,उमा,0.000007,0
26566,मा,0.000007,0
1542,अमा,0.000005,0
17698,नमः,0.000005,0


## Implementing Text Autocorrection Feature from Scratch
Reference: Coursera NLP Specialization Course

Function to get frequency of each word

In [67]:
def get_frequency_of_words(words):
    word_count_dict = {}
    for word in words:
        if word in word_count_dict:
            word_count_dict[word] += 1
        else:
            word_count_dict[word] = 1 
    return word_count_dict

word_count_dict = get_frequency_of_words(words)
print(f"There are {len(word_count_dict)} key value pairs")

There are 496265 key value pairs


Function to get probability of each word

In [68]:
def get_probs(word_count_dict):
    probs = {}
    total = sum(word_count_dict.values())
    for key in word_count_dict.keys():
        probs[key] = word_count_dict[key]/total 
    return probs 

Edit Functions

In [69]:
test_word = "नेपाब"

In [70]:
test_word in vocabulary

False

In [71]:
def delete_letter(word):
    '''
    Input:
        word: the string/word for which you will generate all possible words 
                in the vocabulary which have 1 missing character
    Output:
        delete_l: a list of all possible strings obtained by deleting 1 character from word
    '''
    
    delete_l = []
    split_l = []
    
    split_l = [(word[:i],word[i:]) for i in range(len(word)+1)]
    #split_l is of form [(L1,R1),(L2,R2),...]
    delete_l = [L+R[1:] for L,R in split_l if R]

    return  delete_l

delete_word_l = delete_letter(word=test_word)
delete_word_l

['ेपाब', 'नपाब', 'नेाब', 'नेपब', 'नेपा']

In [72]:
# SwitchLetter:swap two adjacent letters
def switch_letter(word):
    '''
    Input:
        word: input string
     Output:
        switches: a list of all possible strings with one adjacent charater switched
    ''' 
    
    switch_l = []
    split_l = []
    
    split_l = [(word[:i],word[i:]) for i in range(len(word)+1)]
    switch_l = [L+R[1]+R[0]+R[2:] for L,R in split_l if len(R) >= 2] 
    
    return switch_l

switch_word_l = switch_letter(word=test_word)
switch_word_l

['ेनपाब', 'नपेाब', 'नेापब', 'नेपबा']

I have only considered these alphabets as unique alphabets but there are more alphabets in Nepali 

In [8]:
alphabets = "ञज्ञघङझछटठडढणत्रधभचतथगषयउसपवजनमकबशहअखदलफर"

In [74]:
# replace_letter: changes one letter to another
def replace_letter(word):
    '''
    Input:
        word: the input string/word 
    Output:
        replaces: a list of all possible strings where we replaced one letter from the original word. 
    ''' 
    
    letters = alphabets
    
    replace_l = []
    split_l = []
    
    split_l = [(word[:i],word[i:]) for i in range(len(word)+1) if len(word[i:]) >= 1]
    replace_l = [L + c + R[1:] if len(R) >= 2 else L+c for L,R in split_l for c in letters]
    replace_set = set(replace_l)
    replace_set.discard(word)
    
    # turn the set back into a list and sort it, for easier viewing
    replace_l = sorted(list(replace_set))
        
    return replace_l

replace_l = replace_letter(word=test_word)
len(replace_l)

177

In [75]:
# insert_letter: adds additional characters
def insert_letter(word):
    '''
    Input:
        word: the input string/word 
    Output:
        inserts: a set of all possible strings with one new letter inserted at every offset
    ''' 
    letters = alphabets
    insert_l = []
    split_l = []
    
    split_l = [(word[:i],word[i:]) for i in range(len(word)+1)]
    insert_l = [L + c + R for L,R in split_l for c in letters]
    
    return insert_l

insert_l = insert_letter(word=test_word)
len(insert_l)

246

In [76]:
def edit_one_letter(word, allow_switches = True):
    """
    Input:
        word: the string/word for which we will generate all possible wordsthat are one edit away.
    Output:
        edit_one_set: a set of words with one possible edit. Please return a set. and not a list.
    """
    
    edit_one_set = set()
    
    edit_one_set = set(delete_letter(word)).union(set(switch_letter(word)), set(replace_letter(word)), set(insert_letter(word)))
    
    # return this as a set and not a list
    return set(edit_one_set)

def edit_two_letters(word, allow_switches = True):
    '''
    Input:
        word: the input string/word 
    Output:
        edit_two_set: a set of strings with all possible two edits
    '''
    
    edit_two_set = set()
    
    tmp_edit_one_set = edit_one_letter(word)
    tmp_edit_one_list = list(tmp_edit_one_set)
    edit_two_list = [edit_one_letter(w) for w in tmp_edit_one_list]
    for s in edit_two_list:
        edit_two_set.update(s)
    
    # return this as a set instead of a list
    return set(edit_two_set)

def get_corrections(word, probs, vocab, n=2):
    '''
    Input: 
        word: a user entered string to check for suggestions
        probs: a dictionary that maps each word to its probability in the corpus
        vocab: a set containing all the vocabulary
        n: number of possible word corrections you want returned in the dictionary
    Output: 
        n_best: a list of tuples with the most probable n corrected words and their probabilities.
    '''
    
    suggestions = []
    n_best = []
    
    #Step 1: create suggestions as described above   
    if word in vocab:
        return [[word,1]]
    else:
        suggestions = list(edit_one_letter(word).intersection(vocab) or edit_two_letters(word).intersection(
                vocab))
    # print(suggestions)
    #Step 2: determine probability of suggestions
    
    #Step 3: Get all your best words and return the most probable top n_suggested words as n_best
    
    n_best = [[s, probs[s]] for s in list(reversed(suggestions))]
    

    return n_best

In [77]:
test_word1 = "नेपा"

In [78]:
def nep_autocorrect(my_word):
    probs = get_probs(word_count_dict)
    tmp_corrections = get_corrections(my_word, probs, vocabulary, 2)
    print(f"Probable words for {my_word} are: ")
    for i, word_prob in enumerate(tmp_corrections):
        print(f"word {i}: {word_prob[0]}, probability {word_prob[1]:.6f}")

In [79]:
my_word = test_word
nep_autocorrect(my_word)

Probable words for नेपाब are: 
word 0: नेपार, probability 0.000002
word 1: नेपाल, probability 0.000020
word 2: नेपा, probability 0.000002


In [80]:
my_word = test_word1
nep_autocorrect(my_word)

Probable words for नेपा are: 
word 0: नेपा, probability 1.000000


Finding Minimum edit distance and number of edits needed to transform the given word to reference word

In [81]:
def min_edit_distance(source, target, ins_cost = 1, del_cost = 1, rep_cost = 2):
    '''
    Input: 
        source: a string corresponding to the string you are starting with
        target: a string corresponding to the string you want to end with
        ins_cost: an integer setting the insert cost
        del_cost: an integer setting the delete cost
        rep_cost: an integer setting the replace cost
    Output:
        D: a matrix of len(source)+1 by len(target)+1 containing minimum edit distances
        med: the minimum edit distance (med) required to convert the source string to the target
    '''
    # use deletion and insert cost as  1
    m = len(source) 
    n = len(target) 
    #initialize cost matrix with zeros and dimensions (m+1,n+1) 
    D = np.zeros((m+1, n+1), dtype=int) 
    
    
    # Fill in column 0, from row 1 to row m, both inclusive
    for row in range(1,m+1): # Replace None with the proper range
        D[row,0] = D[row-1,0] + del_cost
        
    # Fill in row 0, for all columns from 1 to n, both inclusive
    for col in range(1,n+1): # Replace None with the proper range
        D[0,col] = D[0,col-1] + ins_cost
        
    # Loop through row 1 to row m, both inclusive
    for row in range(1,m+1): 
        
        # Loop through column 1 to column n, both inclusive
        for col in range(1,n+1):
            
            # Intialize r_cost to the 'replace' cost that is passed into this function
            r_cost = rep_cost
            
            # Check to see if source character at the previous row
            # matches the target character at the previous column, 
            if source[row-1] == target[col-1]:
                # Update the replacement cost to 0 if source and target are the same
                r_cost = 0
                
            # Update the cost at row, col based on previous entries in the cost matrix
            # Refer to the equation calculate for D[i,j] (the minimum of three calculated costs)
            D[row,col] = min([D[row-1,col]+del_cost, D[row,col-1]+ins_cost, D[row-1,col-1]+r_cost])
          
    # Set the minimum edit distance with the cost found at row m, column n
    med = D[m,n]
    
    return D, med

In [82]:
source =  'नेपाब'
target = 'नेपाल'
matrix, min_edits = min_edit_distance(source, target)
print("minimum edits: ",min_edits, "\n")
idx = list('#' + source)
cols = list('#' + target)
df = pd.DataFrame(matrix, index=idx, columns= cols)
print(df)

minimum edits:  2 

   #  न  े  प  ा  ल
#  0  1  2  3  4  5
न  1  0  1  2  3  4
े  2  1  0  1  2  3
प  3  2  1  0  1  2
ा  4  3  2  1  0  1
ब  5  4  3  2  1  2
